In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('../data/home_final.csv')
X = dataset.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 9, 10]].values
y = dataset.iloc[:, 11].values

In [3]:
# ------ Part-1: Data preprocessing ----------

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

for i in range(8):
    if i == 5:
        continue
        
    labelencoder_X = LabelEncoder()
    X[:, i] = labelencoder_X.fit_transform(X[:, i])

In [4]:
onehotencoder = OneHotEncoder(categorical_features=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
X = onehotencoder.fit_transform(X).toarray()

In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [7]:
# ------- Part-2: Build the ANN --------

# import keras library and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initializing the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
layer_info = Dense(1, activation='relu', kernel_initializer='uniform')
classifier.add(layer_info)

# Adding second hidden layer
layer_info = Dense(1, activation='relu', kernel_initializer='uniform')
classifier.add(layer_info)

# Adding output layer
layer_info = Dense(1, activation='sigmoid', kernel_initializer='uniform')
classifier.add(layer_info)

E:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
# Compiling the ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fitting the ANN to the training set
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
4166/4166 [==============================] - 1s 236us/step - loss: 0.6931 - acc: 0.4930
Epoch 2/100
4166/4166 [==============================] - 0s 76us/step - loss: 0.6525 - acc: 0.6109
Epoch 3/100
4166/4166 [==============================] - 0s 76us/step - loss: 0.5514 - acc: 0.7254
Epoch 4/100
4166/4166 [==============================] - 0s 77us/step - loss: 0.4944 - acc: 0.7703
Epoch 5/100
4166/4166 [==============================] - 0s 75us/step - loss: 0.4672 - acc: 0.7787
Epoch 6/100
4166/4166 [==============================] - 0s 74us/step - loss: 0.4493 - acc: 0.7892
Epoch 7/100
4166/4166 [==============================] - 0s 76us/step - loss: 0.4364 - acc: 0.7969
Epoch 8/100
4166/4166 [==============================] - 0s 75us/step - loss: 0.4272 - acc: 0.8024
Epoch 9/100
4166/4166 [==============================] - 0s 74us/step - loss: 0.4198 - acc: 0.8041
Epoch 10/100
4166/4166 [==============================] - 0s 71us/step - loss: 0.4128 - acc: 0.8111
Epoch 11

4166/4166 [==============================] - 0s 65us/step - loss: 0.3024 - acc: 0.8814
Epoch 83/100
4166/4166 [==============================] - 0s 63us/step - loss: 0.3001 - acc: 0.8821
Epoch 84/100
4166/4166 [==============================] - 0s 63us/step - loss: 0.2986 - acc: 0.8836
Epoch 85/100
4166/4166 [==============================] - 0s 63us/step - loss: 0.2986 - acc: 0.8836
Epoch 86/100
4166/4166 [==============================] - 0s 63us/step - loss: 0.2988 - acc: 0.8838
Epoch 87/100
4166/4166 [==============================] - 0s 62us/step - loss: 0.2976 - acc: 0.8843
Epoch 88/100
4166/4166 [==============================] - 0s 63us/step - loss: 0.2976 - acc: 0.8841
Epoch 89/100
4166/4166 [==============================] - 0s 63us/step - loss: 0.2971 - acc: 0.8845
Epoch 90/100
4166/4166 [==============================] - 0s 62us/step - loss: 0.2976 - acc: 0.8841
Epoch 91/100
4166/4166 [==============================] - 0s 63us/step - loss: 0.2975 - acc: 0.8843
Epoch 92/100


In [9]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [10]:
cm

array([[272, 244],
       [ 75, 451]], dtype=int64)